I want to know if we have access to user partisanship for each topic.

In [2]:
import pandas as pd

In [2]:
# check note_info file

note_info = pd.read_parquet('aux_note_info.parquet', engine='auto')

In [3]:
len(note_info)

1946619

In [4]:
note_info.columns

Index(['noteId', 'ratingWeight', 'createdAtMillis', 'noteAuthorParticipantId',
       'awaitingBool', 'numRatingsLast28', 'currentStatus', 'crhBool',
       'crnhBool', 'unlockedRatingStatus', 'helpfulOther',
       'helpfulInformative', 'helpfulClear', 'helpfulEmpathetic',
       'helpfulGoodSources', 'helpfulUniqueContext', 'helpfulAddressesClaim',
       'helpfulImportantContext', 'helpfulUnbiasedLanguage', 'notHelpfulOther',
       'notHelpfulIncorrect', 'notHelpfulSourcesMissingOrUnreliable',
       'notHelpfulOpinionSpeculationOrBias', 'notHelpfulMissingKeyPoints',
       'notHelpfulOutdated', 'notHelpfulHardToUnderstand',
       'notHelpfulArgumentativeOrBiased', 'notHelpfulOffTopic',
       'notHelpfulSpamHarassmentOrAbuse', 'notHelpfulIrrelevantSources',
       'notHelpfulOpinionSpeculation', 'notHelpfulNoteNotNeeded',
       'notHelpfulOtherAdjusted', 'notHelpfulIncorrectAdjusted',
       'notHelpfulSourcesMissingOrUnreliableAdjusted',
       'notHelpfulOpinionSpeculationOrBi

In [5]:
# check helpfulness scores

h_scores = pd.read_parquet('helpfulness_scores.parquet', engine='auto')

In [6]:
h_scores.columns

Index(['raterParticipantId', 'coreRaterIntercept', 'coreRaterFactor1',
       'crhCrnhRatioDifference', 'meanNoteScore', 'raterAgreeRatio',
       'successfulRatingHelpfulCount', 'successfulRatingNotHelpfulCount',
       'successfulRatingTotal', 'unsuccessfulRatingHelpfulCount',
       'unsuccessfulRatingNotHelpfulCount', 'unsuccessfulRatingTotal',
       'ratingsAwaitingMoreRatings', 'ratedAfterDecision',
       'notesCurrentlyRatedHelpful', 'notesCurrentlyRatedNotHelpful',
       'notesAwaitingMoreRatings', 'enrollmentState',
       'successfulRatingNeededToEarnIn', 'authorTopNotHelpfulTagValues',
       'timestampOfLastStateChange', 'aboveHelpfulnessThreshold',
       'isEmergingWriter', 'aggregateRatingReceivedTotal',
       'timestampOfLastEarnOut', 'groupRaterIntercept', 'groupRaterFactor1',
       'modelingGroup', 'raterHelpfulnessReputation', 'numberOfTimesEarnedOut',
       'expansionRaterIntercept', 'expansionRaterFactor1',
       'expansionPlusRaterIntercept', 'expansionPlus

## Monthly count of unique raters

In [6]:
# prepare for load and keeping aggregated data
chunk_size = 1000000
# this will hold the cumulative grouped result
monthly_count = pd.DataFrame(columns=['year', 'month', 'unique_id_count'])


In [9]:
# load rating file in chunks
# there are 20 rating files, some ~2.5 GB. Since I can only keep them one at a time I
# change the file name one by one
rating_file = 'ratings/ratings-00001.tsv'
with pd.read_csv(rating_file, sep='\t', chunksize=chunk_size) as reader:
    for chunk in reader:
        
        # add needed columns to this chunk
        chunk['createdAt'] = pd.to_datetime(chunk['createdAtMillis'], unit='ms')
        chunk['year'] = chunk['createdAt'].dt.year
        chunk['month'] = chunk['createdAt'].dt.month
        grouped = chunk.groupby(['year', 'month'])['raterParticipantId'].nunique().reset_index(name='unique_id_count')
        # merge with the running monthly_count
        monthly_count = pd.merge(
            monthly_count, grouped,
            on=['year', 'month'], how='outer', suffixes=('', '_new')
        )

        # sum counts (fillna for missing)
        monthly_count['unique_id_count'] = (
            monthly_count['unique_id_count'].fillna(0)
            + monthly_count['unique_id_count_new'].fillna(0)
        )

        # drop the temporary column
        monthly_count = monthly_count.drop(columns=['unique_id_count_new'])


In [10]:
monthly_count

,year,month,unique_id_count
0,2021,1,52.0
1,2021,2,125.0
2,2021,3,128.0
3,2021,4,110.0
4,2021,5,115.0
5,2021,6,356.0
6,2021,7,342.0
7,2021,8,457.0
8,2021,9,711.0
9,2021,10,741.0
